In [18]:
import pandas as pd
import openai
from dotenv import load_dotenv
import os
import ast
from openai import AzureOpenAI
import faiss

import numpy as np
import json

### Load Enviroment files

In [19]:
# Load environment variables
load_dotenv()

True

In [20]:
# OpenAI API configuration
openai.api_type = "azure"
openai.api_key = os.getenv("api_key_azure")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("api_version")
openai_deployment = "sdgi-gpt-35-turbo-16k"


client = AzureOpenAI(
  api_key = os.getenv("api_key_azure"),  
  api_version = os.getenv("api_version"),
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

embedding_model = os.getenv("USER_QUERY_EMBEDDING_ENGINE") 

# print(openai.api_key)
# print(openai.api_base)
# print(openai.api_version)


<h3>globals</h3>

In [21]:
test_query="What are the sustainable energy priorities for UNDP?"


<h3> helper functions </h3>

In [22]:
# use this function to make simple openAI Calls
def callOpenAI(prompt):  
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=[
                        {"role": "user", "content": prompt},
                    ]
                )
    response = response_entities.choices[0].message.content
    return response


<h3> processing modules </h3>

In [23]:
## module to create prompt engineering template  

def promptEngineeringModule(user_query):
    
    ## generate prompt engineering text and template
    llm_instructions="llm instruction template here, with placeholders for insertion of user query, excerpts, indicator data, and entity and relation info" 
    
    return llm_instructions

# test usage
llm_instructions=promptEngineeringModule(test_query)
print(llm_instructions)

llm instruction template here, with placeholders for insertion of user query, excerpts, indicator data, and entity and relation info


In [24]:
def extractEntitiesFromQuery(user_query):
    prompt = f"""
    Extract entities from the following user query: \"{user_query}\" and return output in array format e.g ['entity1','entity2']
    
    -Entities should be directly related to the domain or topic of interest. They should represent important concepts that contribute to the understanding of the subject matter.
    -Each entity in the knowledge graph should be distinct and have a unique identifier. This ensures clarity and avoids ambiguity when establishing relationships between entities.
    """
    entity_list = callOpenAI(prompt)   
    return entity_list

# Test usage
test_query = "What are the sustainable energy for UNDP?"
entity_list = extractEntitiesFromQuery(test_query)
print(entity_list)

['sustainable energy', 'UNDP']


In [25]:
## module to get information on the entities from user query using the KG
def knowledgeGraphModule(user_query):
    
    # generate list of entities based on user query
    entity_list = extractEntitiesFromQuery(user_query)
    my_list = ast.literal_eval(entity_list)
    prompt_summarise_entites = f"""
    Summarize all relations between all the entities : {my_list}
    """
    summarise_entities = callOpenAI(prompt_summarise_entites)
    # Initialize an empty dictionary to store information
    entities_dict = {
        "relations": summarise_entities,
        "entities": {}
    }
    # Loop through each entity in the list
    for entity in my_list:
        # Fetch information about the entity from your knowledge graph
        prompt = f"Give me a short description 50 words of {entity}"
        entity_info = callOpenAI(prompt)
        # Add the entity information to the dictionary
        entities_dict["entities"][entity] = entity_info
    
    return entities_dict


# Test usage
test_query = "what is the major work on UNDP in Afganistan?"
entities_dict = knowledgeGraphModule(test_query)
print(entities_dict)

{'relations': 'The relation between UNDP and Afghanistan is that UNDP works in Afghanistan to support development projects and initiatives.', 'entities': {'UNDP': 'The United Nations Development Programme (UNDP) is a global organization that works to eradicate poverty, reduce inequalities, and promote sustainable development. It provides support to countries in areas such as governance, climate change, and crisis response, aiming to improve the lives of people and protect the planet.', 'Afghanistan': 'Afghanistan is a landlocked country located in Central Asia, bordered by Iran, Pakistan, Turkmenistan, Uzbekistan, Tajikistan, and China. It is known for its rugged mountainous terrain, rich cultural heritage, and complex history. Despite facing numerous challenges, Afghanistan is home to resilient people, diverse ethnic groups, and a vibrant blend of traditions.'}}


In [26]:
def search_embeddings(user_query):
    df = pd.read_pickle('../models/df_embed_EN.pkl')
    df_filtered = df
    length = len(df_filtered.head())
    filtered_embeddings_arrays = np.array(list(df_filtered['Embedding']))
    index = faiss.IndexFlatIP(filtered_embeddings_arrays.shape[1]) 
    index.add(filtered_embeddings_arrays)
    
    user_query_embedding = client.embeddings.create( 
        input=user_query ,model= embedding_model
    ).data[0].embedding
    
    k = min(5, length)
    distances, indices = index.search(np.array([user_query_embedding]), k)
    return df_filtered, distances, indices

In [27]:
def get_answer(user_question, content):
    system_prompt = "You are a system that answers user questions based on excerpts from PDF documents provided for context. Only answer if the answer can be found in the provided context. Do not make up the answer; if you cannot find the answer, say so."
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_question},
        {'role': 'user', 'content': content},
    ]
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0.2,
                    messages=messages
                )
    response = response_entities.choices[0].message.content
    return response
  

In [28]:
def map_to_structure(qs):
    result_dict = {}

    # Extract the DataFrame from the tuple
    dataframe = qs[0]

    # Counter to limit the loop to 10 iterations
    count = 0

    for index, row in dataframe.iterrows():
        # Define a unique identifier for each document, you can customize this based on your data
        document_id = f"doc-{index + 1}"
        # Handle NaN in content by using fillna
        content = row["Content"]
        content = ' '.join(row["Content"].split()[:160])

        # Create a dictionary for each document
        document_info = {
            "title": row["Document Title"],
            "extract": content or "",  # You may need to adjust this based on your column names
            "category": row["Category"],
            "link": row["Link"],
            "thumbnail": ''
        }
        # print(document_info)
        # Add the document to the result dictionary
        result_dict[document_id] = document_info

        # # Increment the counter
        # count += 1

        # # Break out of the loop if the counter reaches 10
        # if count == 1:
        #     break

    return result_dict


In [29]:
## module to extract text from documents and return the text and document codes

def semanticSearchModule(user_query):
    qs = search_embeddings(user_query) #df, distances, indices
    result_structure = map_to_structure(qs)
    return result_structure

#test usage
excerpts_dict=semanticSearchModule(test_query)
# print(excerpts_dict)



# Define the filename to save the JSON data -  can remove later
json_filename = "excerpts.json"

# Save excerpts_dict to the JSON file just for a better preview
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(excerpts_dict, json_file, ensure_ascii=False, indent=4)

print(f"Excerpts saved to {json_filename}")

Excerpts saved to excerpts.json


In [30]:
## module to get data for specific indicators which are identified is relevant to the user query

def indicatorsModule(user_query): #lower priority
    
    # find relevant indicators based on uesr query and extract values
    indicators_dict={
        "indicator-id-1":"value from indicator-id-1",
        "indicator-id-2":"value from indicator-id-2"
    }#temp
    
    return indicators_dict

#test usage
indicators_dict=indicatorsModule(test_query)
print(indicators_dict)

{'indicator-id-1': 'value from indicator-id-1', 'indicator-id-2': 'value from indicator-id-2'}


In [31]:
## module to generate query ideas

def queryIdeationModule(user_query): # lower priority
    
    ## generate list of prompt ideas based on user query
    query_idea_list=["prompt idea 1","prompt idea 2","prompt idea 3"]#temp
    
    return query_idea_list

#test usage
query_idea_list=queryIdeationModule(test_query)
print(query_idea_list)

['prompt idea 1', 'prompt idea 2', 'prompt idea 3']


<h3> synthesis module </h3>

In [32]:
# module to synthesize answer using retreival augmented generation approach

def synthesisModule(user_query, llm_instructions, entities_dict, excerpts_dict, indicators_dict):
    
    ###synthesize data into structure within llm prompt engineering instructions
    answer="test answer"
    
    return answer  

## to test this, run the full pipeline with the handleApiCall function

<h3> run pipeline </h3>

In [33]:
# full pipeline with retreival, synthesis of answer to user query, and structure results into api response

def handleApiCall(user_query):
        
    ##run processing modules (in parallel)
    llm_instructions=promptEngineeringModule(user_query)
    entities_dict=knowledgeGraphModule(user_query)
    excerpts_dict=semanticSearchModule(user_query)
    indicators_dict=indicatorsModule(user_query) ##lower priority
    query_idea_list=queryIdeationModule(user_query) ##lower priority
    
    ##synthesis module
    answer=synthesisModule(user_query, llm_instructions, entities_dict, excerpts_dict, indicators_dict)
       
    ##structure response
    response={
        "user_query":user_query,
        "answer":answer,
        "sources":excerpts_dict,
        "query_ideas":query_idea_list,
        "entities":list(entities_dict["entities"].keys())       
    }
    
    return response

# test usage
response=handleApiCall(test_query) 
# print(response)

<h3>testing</h3>

In [ ]:
## next step, develop automated testing for all modules
## iterate through test_queries and build automated tests to score results

# open testing dataset with queries and expected results
test_queries_df=pd.read_csv("../testing/energy_ai_test_dataset_v0.csv")

